In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from scipy import signal, linalg
from tqdm import tqdm
import transforms3d as t3d
import cv2

from src.load_data import load_all_data
from utils import *
from src.map import MAP

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
dataset = 20
data = load_all_data(dataset)
dir_path = os.path.join(os.getcwd(), "data/dataRGBD/")

In [8]:
robot_trajectory = np.load('deadreckoning-trajectory.npy')
robot_theta = np.load('deadreckoning-theta.npy')

In [9]:
camera_R = t3d.euler.euler2mat(0,0.36,0.021)
camera_p_body_frame = np.array([0.16766, 0, 0.38])
K = np.array([[585.05108211, 0, 242.94140713], [0, 585.05108211, 315.83800193], [0, 0, 1]])
R_o_r = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]])
lidar_position_wrt_body = np.array([0.13673, 0, 0.51435])

In [10]:
data.disp_stamps.shape, data.rgb_stamps.shape

((2407,), (2289,))

In [11]:
occupancy_map = np.load("dead-occupancy-map.npy")

In [15]:
binary_map = np.zeros_like(occupancy_map)
binary_map[occupancy_map == 100] = 0
binary_map[occupancy_map == -100] = 1

num_rgb_imgs = data.rgb_stamps.shape[0]

resolution = 0.01
xmin = -15
ymin = -15
xmax = 30
ymax = 30

mp = MAP(resolution, xmin, ymin, xmax, ymax)
mp.convert_to_color()
frames = []
for img_idx in tqdm(range(num_rgb_imgs)):
    rgb_path = os.path.join(dir_path, f"RGB{dataset}/rgb{dataset}_{img_idx+1}.png")
    rgb_img = cv2.imread(rgb_path)
    rgb_img = rgb_img[...,::-1]

    rgb_timestamp = data.rgb_stamps[img_idx]
    disp_idx = np.argmin(np.abs(data.disp_stamps - rgb_timestamp)) + 1

    disp_path = os.path.join(dir_path, f"Disparity{dataset}/disparity{dataset}_{disp_idx}.png")
    disp_img = cv2.imread(disp_path, cv2.IMREAD_UNCHANGED)

    # convert from disparity from uint16 to double
    disparity = disp_img.astype(np.float32)

    # get depth
    dd = (-0.00304 * disparity + 3.31)
    z = 1.03 / dd


    # calculate u and v coordinates 
    v,u = np.mgrid[0:disparity.shape[0],0:disparity.shape[1]]
    #u,v = np.meshgrid(np.arange(disparity.shape[1]),np.arange(disparity.shape[0]))

    # get 3D coordinates 
    fx = 585.05108211
    fy = 585.05108211
    cx = 315.83800193
    cy = 242.94140713
    x = (u-cx) / fx * z
    y = (v-cy) / fy * z

    # calculate the location of each pixel in the RGB image
    rgbu = np.round((u * 526.37 + dd*(-4.5*1750.46) + 19276.0)/fx)
    rgbv = np.round((v * 526.37 + 16662.0)/fy)
    rgbu = rgbu.astype(np.int32)
    rgbv = rgbv.astype(np.int32)

    x = x.reshape(-1)
    y = y.reshape(-1)
    z = z.reshape(-1)

    optical_coordinates = np.stack([x,y,z])

    real_coordinates_camera = np.linalg.inv(R_o_r) @ optical_coordinates.reshape((3,-1))
    coordinates_body = camera_R @ real_coordinates_camera + camera_p_body_frame[:, None]

    r_idx = np.argmin(np.abs(data.lidar_stamps - rgb_timestamp))
    
    body_p = robot_trajectory[:, r_idx]
    body_p = np.hstack((body_p, 0))
    
    R_p = get_rotation_matrix(robot_theta[:, r_idx])
    R_p = np.pad(R_p, (0, 1), 'constant', constant_values=(0))
    R_p[-1,-1] = 1

    coordinates_world = R_p @ coordinates_body + body_p[:, None]
    xis, yis = mp.get_pixel_coordinates(coordinates_world)

    # Get only those indexes whose z-coordinate is lower than lidar
    valid = coordinates_world[2,:] <= lidar_position_wrt_body[2]

    # Filter wrt to z
    xis = xis[valid]
    yis = yis[valid]
    rgbu = rgbu.reshape(-1)[valid]
    rgbv = rgbv.reshape(-1)[valid]

    # Now filter wrt to valid rgbu, rgbv values
    valid = (rgbu>= 0)&(rgbu < disparity.shape[1])&(rgbv>=0)&(rgbv<disparity.shape[0])
    xis = xis[valid]
    yis = yis[valid]
    rgbu = rgbu[valid]
    rgbv = rgbv[valid]


    mp.MAP["map"][xis, yis, :] = rgb_img[rgbv, rgbu, :]
    color_map = mp.MAP["map"] * binary_map[:,:,None]
    frames.append(color_map)
        

mp.MAP["map"] = mp.MAP["map"].astype(np.int16)


  1%|▏         | 31/2289 [00:19<23:54,  1.57it/s]  


KeyboardInterrupt: 

In [ ]:
np.save("texture.npy", mp.MAP["map"])

In [ ]:
savepath = os.path.join(os.getcwd(), "results/dataset20/texture/")
idx = 0
for frm in tqdm(frames):
    idx += 1
    plt.imsave(savepath + f"{idx*100}.png", frm.astype(np.uint8))

In [ ]:
binary_map = np.zeros_like(occupancy_map)
binary_map[occupancy_map == 100] = 0
binary_map[occupancy_map == -100] = 1

In [ ]:
plt.imshow(binary_map)

In [ ]:
tmap = mp.MAP["map"] * binary_map[:,:, None]
tmap = tmap.astype(np.int16)

plt.imshow(tmap)

In [ ]:
plt.imshow(mp.MAP["map"].astype(np.int16))
plt.savefig("text.png")

In [ ]:
data.disp_stamps.shape, data.rgb_stamps.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from IPython.display import clear_output
color_grid = np.zeros([1000,1000,3],dtype='uint8')


disp_path = "/home/vatsalya/UCSD/ECE276A_PR2/data/dataRGBD/Disparity20/"
rgb_path = "/home/vatsalya/UCSD/ECE276A_PR2/data/dataRGBD/RGB20/"

def normalize(img):
    max_ = img.max()
    min_ = img.min()
    return (img - min_)/(max_-min_)


for t in range(data.rgb_stamps.shape[0]-1): 
# load RGBD image
  #rgb_idx = time_index_encoder_rgb[encoder_stamps[t]]
  disp_idx = time_index_rgb_disp[rgb_stamps[t]]+1
  
  imd = cv2.imread(disp_path+'disparity{}_{}.png'.format(20,disp_idx),cv2.IMREAD_UNCHANGED) # (480 x 640)
  imc = cv2.imread(rgb_path+'rgb{}_{}.png'.format(20, t+1))[...,::-1] # (480 x 640 x 3)
  #print(imc.shape)
  r, g, b = cv2.split(imc)

  # convert from disparity from uint16 to double
  disparity = imd.astype(np.float32)

  # get depth
  dd = (-0.00304 * disparity + 3.31)
  z = 1.03 / dd


  # calculate u and v coordinates 
  v,u = np.mgrid[0:disparity.shape[0],0:disparity.shape[1]]
  #u,v = np.meshgrid(np.arange(disparity.shape[1]),np.arange(disparity.shape[0]))

  # get 3D coordinates 
  fx = 585.05108211
  fy = 585.05108211
  cx = 315.83800193
  cy = 242.94140713
  x = (u-cx) / fx * z
  y = (v-cy) / fy * z

  # calculate the location of each pixel in the RGB image
  rgbu = np.round((u * 526.37 + dd*(-4.5*1750.46) + 19276.0)/fx)
  rgbv = np.round((v * 526.37 + 16662.0)/fy)
  valid = (rgbu>= 0)&(rgbu < disparity.shape[1])&(rgbv>=0)&(rgbv<disparity.shape[0])

  rows = rgbu.shape[0]
  cols = rgbv.shape[1]

  rgbu = rgbu.reshape(-1)
  rgbv = rgbv.reshape(-1)
  z=z.reshape(-1)
  u = rgbu * z
  v = rgbv * z
  image_mat = np.vstack((u,v,z))

  # cam_frame_coords = np.apply_along_axis(pixel_to_cam,0,image_mat)
  # body_frame_coords = np.apply_along_axis(pix_cam_to_body,0,cam_frame_coords)
  # world_frame_coords = np.apply_along_axis(pix_body_to_world,0,body_frame_coords,np.array([0,0,0]))

  cam_frame_coords = pixel_to_cam(image_mat)
  body_frame_coords = pix_cam_to_body(cam_frame_coords)
  world_frame_coords = pix_body_to_world(body_frame_coords,best_pose_estimate[:,time_index_rgb_encoder[rgb_stamps[t]]-1])
  # # print(world_frame_coords[:,0])

  X_world = world_frame_coords[0,:]
  Y_world = world_frame_coords[1,:]
  Z_world = world_frame_coords[2,:]

  ind_thres = np.where(Z_world <=0.5)
  Y_world = Y_world[ind_thres]
  X_world = X_world[ind_thres]
  #Y_world = Y_world


  x_cell_idx = (600 - (X_world//0.09)).astype(int)
  y_cell_idx = (600 - (Y_world//0.09)).astype(int)
  #print(x_cell_idx)

  r = r.reshape([rows*cols,1])[ind_thres]
  g = g.reshape([rows*cols,1])[ind_thres]
  b = b.reshape([rows*cols,1])[ind_thres]

  # r = r.reshape([rows*cols,1])
  # g = g.reshape([rows*cols,1])
  # b = b.reshape([rows*cols,1])

  #print(x_cell_idx.shape)


  color_grid[x_cell_idx,y_cell_idx,:] = np.hstack([r,g,b]) 

  if t%50 ==0:
     clear_output()
     plt.imshow(color_grid)
     plt.show()